In [13]:
#%cd ..
%run Untitled.ipynb
#%cd BestModels

# Dataset

Download the dataset, divide it in train, validation and test sets and then define the dataloaders.

In [14]:
forest_data = fetch_covtype()

In [15]:
forest_key = list(forest_data.keys())

In [16]:
forest_key

['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR']

In [17]:
torch.from_numpy(forest_data['data']).shape, forest_data['target'].shape

(torch.Size([581012, 54]), (581012,))

In [18]:
batch_forest = 32

# Split train-validation-test sets 
forest_x_train, forest_x_test, forest_y_train, forest_y_test = train_test_split(forest_data['data'], forest_data['target'], 
                                                                                test_size = 0.2, shuffle = True, random_state=1234)
forest_x_train, forest_x_valid, forest_y_train, forest_y_valid = train_test_split(forest_x_train, forest_y_train, 
                                                                                test_size = 0.25, shuffle = True, random_state=1234)
# Create datasets, I don't drop the last in train set cause is of 30 instead of 32
forest_train = DataLoader(MyDataSet(forest_x_train, forest_y_train, forest_data), batch_size = batch_forest, shuffle = True)
forest_valid = DataLoader(MyDataSet(forest_x_valid, forest_y_valid, forest_data), batch_size = batch_forest, shuffle = True)
forest_test = DataLoader(MyDataSet(forest_x_test, forest_y_test, forest_data), batch_size = batch_forest, shuffle = True)

# Initialization

Here we fix the weight initialization and the random weights that we will need for a network.


In [19]:
forest_net = MLP_forest()

In [20]:
# INITIALIZATION STUFF FOR THE NETWORKS

# Save the initial weight configuration that will be used to train all the networks
torch.save(forest_net.state_dict(), 'weight_initialization.pth')

# Select the random weights to train and save their index in the various linear layer
#Create ad array to store the 2 index of each weight 
memory = np.zeros((2040,2), dtype=np.int_)#3))

#Select the random weights which are trainable
rand_weight = np.random.randint(0,232990, size = 2040, dtype = np.int_)

res = 0
layers = ['1','2','3','4','out']
dim_layer = [(500,54), (300,500), (150,300), (70,150), (7,70)]
num_weight_lin = []

#Find the indexes that identify the selected weights 
for i in range(5):
    dim1,dim2 = dim_layer[i]
    mask = np.clip(rand_weight-(dim1*dim2), a_min = None,a_max = 0).astype(np.bool_) # mask to select the weights belonging to a layer
    layer_weight = rand_weight[mask] # Select the weight that belong to that layer
    num_weight_lin.append(len(layer_weight))
    
    # save their index position
    memory[res : res+num_weight_lin[-1], 0] = layer_weight//dim2
    memory[res : res+num_weight_lin[-1], 1] = layer_weight%dim2

    #Delete the weights belonging to the first linear layer
    rand_weight = rand_weight[(mask-1).astype(np.bool_)] -(dim1*dim2)

    #Save the last weight entrance that we update
    res += num_weight_lin[-1]

torch.save((memory, num_weight_lin), 'memory.pth')

# Architecture 
The architecutre is the same for all the networks but the trainable weights change. Actually the linear layer in the random weights network architecture are costum, but the only difference that for the random trainable parameters we add a weight that is trainable while the typical weight matrix of the pytorch linear layer is freezed.

## All parameters are trainable

In [ ]:
forest_net = MLP_forest()

In [8]:
# Initialize the weights and check if it is done correctly
InitializeWeight(forest_net)
HaveSameWeight(forest_net, 'weight_initialization.pth', 'cpu', is_dict=True)

lin1.weight True
lin2.weight True
lin3.weight True
lin4.weight True
linout.weight True
bn1.weight True
bn1.bias True
bn1.running_mean True
bn1.running_var True
bn1.num_batches_tracked True
bn2.weight True
bn2.bias True
bn2.running_mean True
bn2.running_var True
bn2.num_batches_tracked True
bn3.weight True
bn3.bias True
bn3.running_mean True
bn3.running_var True
bn3.num_batches_tracked True
bn4.weight True
bn4.bias True
bn4.running_mean True
bn4.running_var True
bn4.num_batches_tracked True


In [9]:
# Check that all the paramters are trainable 
for param in forest_net.parameters():
    print(param.requires_grad)

True
True
True
True
True
True
True
True
True
True
True
True
True


In [21]:
num_epochs = 200
rand = False # Set to True when we train the random weights network
forest_len_val = forest_y_valid.shape[0]
forest_len_train = forest_y_train.shape[0]
forest_len_test = forest_y_test.shape[0]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}') 

loss_func = nn.CrossEntropyLoss()
#optimizer_forest = optim.Adam(forest_net.parameters(), lr = 0.001)
#optimizer_forest = optim.SGD(forest_net.parameters(), lr = 0.001, momentum = 0.5)#, weight_decay = 0.0001)
optimizer_forest = optim.AdamW(forest_net.parameters(), lr = 0.001, weight_decay = 0.0001)
#scheduler = optim.lr_scheduler.StepLR(optimizer_forest, step_size = 50, gamma = 0.1, verbose = True)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_forest, verbose = True)

save_best_model = SaveBestModel()


Using device: cpu


In [ ]:
# Forest Training 

forest_train_loss = []
forest_val_loss = []
forest_val_acc = []
forest_net.to(device)
    
tqdm_bar = tqdm(range(1, num_epochs+1), desc="epoch [loss: ...]")
for epoch in tqdm_bar:
    
    forest_net.train()
    train_loss_averager = make_averager()
    
    batch_bar =  tqdm(forest_train, leave=False, desc='batch', total=len(forest_train))
    for tr_inputs, tr_classes in batch_bar:
        
        loss = Train(optimizer_forest, tr_inputs, tr_classes, device, forest_net, loss_func, rand)
        refresh_bar(batch_bar, f"train batch [loss: {train_loss_averager(loss.item()):.3f}]")
        
    forest_train_loss.append(train_loss_averager(None))
    refresh_bar(tqdm_bar, f"epoch [loss: {train_loss_averager(None):.3f}]")
    
    
    # Validation
    forest_net.eval()
    sum_misclassified = 0
    val_loss_averager = make_averager()
    
    with torch.no_grad():
        for inputs, classes in forest_valid:
            
            loss, misclassified = Validation(inputs, classes, device, forest_net, loss_func)
            sum_misclassified += misclassified
            val_loss_averager(loss.item())
            
        val_acc = 1 - sum_misclassified/forest_len_val
        forest_val_loss.append(val_loss_averager(None))
        forest_val_acc.append(val_acc)
    
    save_best_model(forest_val_loss[-1], epoch, forest_net, optimizer_forest, loss_func, forest_train_loss[-1], forest_val_acc[-1])
    #scheduler.step()
    #lr_schedule(epoch)
    
    print(f'Epoch: {epoch} \t Training loss: {forest_train_loss[-1]:.3f} \tValidation loss: {forest_val_loss[-1]:.3f} \tValidation accuracy: {val_acc:.3f}')    
    

In [ ]:
plot_loss(forest_train_loss, title= "Train loss", axis = "Loss")

In [ ]:
plot_loss(forest_val_loss, title= "Validation loss", axis = "Loss")

In [ ]:
plot_loss(forest_val_acc, title= "Validation", axis = "Accuracy")

In [25]:
# Test

forest_net.eval()
test_loss_averager = make_averager()
sum_misclassified = 0
forest_net.to(device)

with torch.no_grad():
    batch_bar =  tqdm(forest_valid, leave=False, desc='batch', total=len(forest_valid))
    for inputs, classes in batch_bar:
        loss, misclassified = Validation(inputs, classes, device, forest_net, loss_func)
        sum_misclassified += misclassified
        test_loss_averager(loss.item())

    test_acc = 1 - sum_misclassified/forest_len_test
    forest_test_loss = test_loss_averager(None)
print(f"Accuracy: {test_acc} \tTest Loss: {forest_test_loss}")

batch:   0%|          | 0/3632 [00:00<?, ?it/s]

Accuracy: 0.12334448099136353 	Test Loss: 7.876975824982584


In [ ]:
name = '24'
with open(f'train_loss{name}.pkl', 'wb') as f:
    pickle.dump(ConvList(forest_train_loss), f)

with open(f'val_loss{name}.pkl', 'wb') as f1:
    pickle.dump(ConvList(forest_val_loss), f1)

with open(f'val_acc{name}.pkl', 'wb') as f2:
    pickle.dump(ConvList(forest_val_acc), f2)

#with open(f'test{name}.pkl', 'wb') as f3:
 #   pickle.dump(ConvList([test_loss, test_acc] + y_true.to('cpu') + y_score.to('cpu')), f3)

## Only the BN parameters are trainable

In [5]:
forest_net = MLP_forest()

In [6]:
# Initialize the weights and check if it is done correctly
InitializeWeight(forest_net)
HaveSameWeight(forest_net, 'weight_initialization.pth', 'cpu', is_dict=True)

lin1.weight True
lin2.weight True
lin3.weight True
lin4.weight True
linout.weight True
bn1.weight True
bn1.bias True
bn1.running_mean True
bn1.running_var True
bn1.num_batches_tracked True
bn2.weight True
bn2.bias True
bn2.running_mean True
bn2.running_var True
bn2.num_batches_tracked True
bn3.weight True
bn3.bias True
bn3.running_mean True
bn3.running_var True
bn3.num_batches_tracked True
bn4.weight True
bn4.bias True
bn4.running_mean True
bn4.running_var True
bn4.num_batches_tracked True


In [7]:
# Only the BN parameters are trainable 
for param in forest_net.parameters():
    param.requires_grad = False
for param in forest_net.bn1.parameters():
    param.requires_grad = True
for param in forest_net.bn2.parameters():
    param.requires_grad = True
for param in forest_net.bn3.parameters():
    param.requires_grad = True
for param in forest_net.bn4.parameters():
    param.requires_grad = True


In [8]:
# Check that all the paramters are trainable 
for param in forest_net.parameters():
    print(param.requires_grad)

False
False
False
False
False
True
True
True
True
True
True
True
True


In [22]:
num_epochs = 200
rand = False # Set to True when we train the random weights network
forest_len_val = forest_y_valid.shape[0]
forest_len_train = forest_y_train.shape[0]
forest_len_test = forest_y_test.shape[0]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}') 

loss_func = nn.CrossEntropyLoss()
#optimizer_forest = optim.Adam(forest_net.parameters(), lr = 0.001)
#optimizer_forest = optim.SGD(forest_net.parameters(), lr = 0.001, momentum = 0.5)#, weight_decay = 0.0001)
optimizer_forest = optim.AdamW(forest_net.parameters(), lr = 0.001, weight_decay = 0.0001)
#scheduler = optim.lr_scheduler.StepLR(optimizer_forest, step_size = 50, gamma = 0.1, verbose = True)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_forest, verbose = True)

save_best_model = SaveBestModel()


Using device: cpu


In [ ]:
# Forest Training

forest_train_loss = []
forest_val_loss = []
forest_val_acc = []
forest_net.to(device)

tqdm_bar = tqdm(range(1, num_epochs+1), desc="epoch [loss: ...]")
for epoch in tqdm_bar:

    forest_net.train()
    train_loss_averager = make_averager()

    batch_bar =  tqdm(forest_train, leave=False, desc='batch', total=len(forest_train))
    for tr_inputs, tr_classes in batch_bar:

        loss = Train(optimizer_forest, tr_inputs, tr_classes, device, forest_net, loss_func, rand)
        refresh_bar(batch_bar, f"train batch [loss: {train_loss_averager(loss.item()):.3f}]")

    forest_train_loss.append(train_loss_averager(None))
    refresh_bar(tqdm_bar, f"epoch [loss: {train_loss_averager(None):.3f}]")


    # Validation
    forest_net.eval()
    sum_misclassified = 0
    val_loss_averager = make_averager()

    with torch.no_grad():
        for inputs, classes in forest_valid:

            loss, misclassified = Validation(inputs, classes, device, forest_net, loss_func)
            sum_misclassified += misclassified
            val_loss_averager(loss.item())

        val_acc = 1 - sum_misclassified/forest_len_val
        forest_val_loss.append(val_loss_averager(None))
        forest_val_acc.append(val_acc)

    save_best_model(forest_val_loss[-1], epoch, forest_net, optimizer_forest, loss_func, forest_train_loss[-1], forest_val_acc[-1])
    scheduler.step()
    #lr_schedule(epoch)

    print(f'Epoch: {epoch} \t Training loss: {forest_train_loss[-1]:.3f} \tValidation loss: {forest_val_loss[-1]:.3f} \tValidation accuracy: {val_acc:.3f}')


In [ ]:
plot_loss(forest_train_loss, title= "Train loss", axis = "Loss")

In [ ]:
plot_loss(forest_val_loss, title= "Validation loss", axis = "Loss")

In [ ]:
plot_loss(forest_val_acc, title= "Validation", axis = "Accuracy")

In [ ]:
# Test

forest_net.eval()
test_loss_averager = make_averager()
sum_misclassified = 0
forest_net.to(device)

with torch.no_grad():
    batch_bar =  tqdm(forest_valid, leave=False, desc='batch', total=len(forest_valid))
    for inputs, classes in batch_bar:
        loss, misclassified = Validation(inputs, classes, device, forest_net, loss_func)
        sum_misclassified += misclassified
        test_loss_averager(loss.item())

    test_acc = 1 - sum_misclassified/forest_len_test
    forest_test_loss = test_loss_averager(None)
print(f"Accuracy: {test_acc} \tTest Loss: {forest_test_loss}")

In [ ]:
def ConvList(l):
    # It puts the list l from CUDA to CPU
    return list(torch.Tensor(l).to('cpu'))

name = 'BN19'
with open(f'train_loss{name}.pkl', 'wb') as f:
    pickle.dump(ConvList(forest_train_loss), f)

with open(f'val_loss{name}.pkl', 'wb') as f1:
    pickle.dump(ConvList(forest_val_loss), f1)

with open(f'val_acc{name}.pkl', 'wb') as f2:
    pickle.dump(ConvList(forest_val_acc), f2)

#with open(f'test{name}.pkl', 'wb') as f3:
 #   pickle.dump(ConvList([test_loss, test_acc] + y_true.to('cpu') + y_score.to('cpu')), f3)

## Only some random wieghts are trainable 

In [9]:
# Define the net initiliaze and check the initialization
forest_net = MLP_forest_RandWeights()
InitializeCheckRandWieght(forest_net)

lin1.weight True
lin2.weight True
lin3.weight True
lin4.weight True
linout.weight True
bn1.weight True
bn1.bias True
bn1.running_mean True
bn1.running_var True
bn1.num_batches_tracked True
bn2.weight True
bn2.bias True
bn2.running_mean True
bn2.running_var True
bn2.num_batches_tracked True
bn3.weight True
bn3.bias True
bn3.running_mean True
bn3.running_var True
bn3.num_batches_tracked True
bn4.weight True
bn4.bias True
bn4.running_mean True
bn4.running_var True
bn4.num_batches_tracked True


In [10]:
# Freeze all the parameters except the random weights

for param in forest_net.parameters():
    param.requires_grad = False

forest_net.lin1.tr_weights.requires_grad = True
forest_net.lin2.tr_weights.requires_grad = True
forest_net.lin3.tr_weights.requires_grad = True
forest_net.lin4.tr_weights.requires_grad = True
forest_net.linout.tr_weights.requires_grad = True


In [11]:
# Check which are freezed
for param in forest_net.parameters():
    print(param.requires_grad)

False
True
False
True
False
True
False
True
False
True
False
False
False
False
False
False
False
False


In [ ]:
num_epochs = 200
rand = True
forest_len_val = forest_y_valid.shape[0]
forest_len_train = forest_y_train.shape[0]
forest_len_test = forest_y_test.shape[0]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

loss_func = nn.CrossEntropyLoss()
optimizer_forest = optim.Adam(forest_net.parameters(), lr = 0.001)
#optimizer_forest = optim.SGD(forest_net.parameters(), lr = 0.001, momentum = 0.5)#, weight_decay = 0.0001)
#optimizer_forest = optim.AdamW(forest_net.parameters(), lr = 0.001, weight_decay = 0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer_forest, step_size = 50, gamma = 0.1, verbose = True)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_forest, verbose = True)

save_best_model = SaveBestModel()


In [ ]:
# Forest Training

forest_train_loss = []
forest_val_loss = []
forest_val_acc = []
forest_net.to(device)

tqdm_bar = tqdm(range(1, num_epochs+1), desc="epoch [loss: ...]")
for epoch in tqdm_bar:

    forest_net.train()
    train_loss_averager = make_averager()

    batch_bar =  tqdm(forest_train, leave=False, desc='batch', total=len(forest_train))
    for tr_inputs, tr_classes in batch_bar:

        loss = Train(optimizer_forest, tr_inputs, tr_classes, device, forest_net, loss_func, rand)
        refresh_bar(batch_bar, f"train batch [loss: {train_loss_averager(loss.item()):.3f}]")

    forest_train_loss.append(train_loss_averager(None))
    refresh_bar(tqdm_bar, f"epoch [loss: {train_loss_averager(None):.3f}]")


    # Validation
    forest_net.eval()
    sum_misclassified = 0
    val_loss_averager = make_averager()

    with torch.no_grad():
        for inputs, classes in forest_valid:

            loss, misclassified = Validation(inputs, classes, device, forest_net, loss_func)
            sum_misclassified += misclassified
            val_loss_averager(loss.item())

        val_acc = 1 - sum_misclassified/forest_len_val
        forest_val_loss.append(val_loss_averager(None))
        forest_val_acc.append(val_acc)

    save_best_model(forest_val_loss[-1], epoch, forest_net, optimizer_forest, loss_func, forest_train_loss[-1], forest_val_acc[-1])
    scheduler.step()
    #lr_schedule(epoch)

    print(f'Epoch: {epoch} \t Training loss: {forest_train_loss[-1]:.3f} \tValidation loss: {forest_val_loss[-1]:.3f} \tValidation accuracy: {val_acc:.3f}')


In [ ]:
plot_loss(forest_train_loss, title= "Train loss", axis = "Loss")

In [ ]:
plot_loss(forest_val_loss, title= "Validation loss", axis = "Loss")

In [ ]:
plot_loss(forest_val_acc, title= "Validation", axis = "Accuracy")

In [60]:
# Test

forest_net.eval()
test_loss_averager = make_averager()
sum_misclassified = 0
forest_net.to(device)
#y_true = torch.Tensor([]).to(device)
#y_score = torch.Tensor([]).to(device)

with torch.no_grad():
    batch_bar =  tqdm(forest_valid, leave=False, desc='batch', total=len(forest_valid))
    for inputs, classes in batch_bar:
        loss, misclassified = Validation(inputs, classes, device, forest_net, loss_func)
        sum_misclassified += misclassified
        test_loss_averager(loss.item())
        
        #Stuff to compute the ROC curve
        #cl = torch.clamp(classes[:,0]-classes[:,1], min =0, max = 1 )
        #y_true = torch.cat((y_true, cl), dim = 0)
        #pred = net(inputs)
        #pr = torch.clamp(pred[:,0]-pred[:,1], min =0, max = 1 )
        #y_score = torch.cat((y_score, pr), dim = 0)

    test_acc = 1 - sum_misclassified/forest_len_test
    forest_test_loss = test_loss_averager(None)
print(f"Accuracy: {test_acc} \tTest Loss: {forest_test_loss}")

batch:   0%|          | 0/3632 [00:00<?, ?it/s]

Accuracy: 0.8092734217643738 	Test Loss: 0.4442348908386829


In [ ]:
def ConvList(l):
    # It puts the list l from CUDA to CPU
    return list(torch.Tensor(l).to('cpu'))

name = 'Rand4'
with open(f'train_loss{name}.pkl', 'wb') as f:
    pickle.dump(ConvList(forest_train_loss), f)

with open(f'val_loss{name}.pkl', 'wb') as f1:
    pickle.dump(ConvList(forest_val_loss), f1)

with open(f'val_acc{name}.pkl', 'wb') as f2:
    pickle.dump(ConvList(forest_val_acc), f2)

#with open(f'test{name}.pkl', 'wb') as f3:
 #   pickle.dump(ConvList([test_loss, test_acc] + y_true.to('cpu') + y_score.to('cpu')), f3)

# Control and compare architectures

In [115]:
train_loss = []
val_loss = []
val_acc = []
y_true = []
y_score = []

for name in ['18', 'BN7', 'Rand4' ]:
    with open(f'train_loss{name}.pkl', 'rb') as f:
        train_loss.append(pickle.load(f))
    
    with open(f'val_loss{name}.pkl', 'rb') as f1:
        val_loss.append(pickle.load(f1))
        
    with open(f'val_acc{name}.pkl', 'rb') as f2:
        val_acc.append(pickle.load(f2))
    
    #with open(f'test{name}.pkl', 'rb') as f3:
    #    [test_loss, test_acc] = pickle.load(f3)
    #    
    #with open(f'test_true{name}.pkl', 'rb') as f3:
    #    y_true.append(pickle.load(f3))
    #
    #with open(f'test_score{name}.pkl', 'rb') as f3:
    #    y_score.append(pickle.load(f3))

In [ ]:
plot_loss_nets(train_loss, name_title = 'Train Loss', name_axis = 'Loss')

In [ ]:
plot_loss_nets(val_loss, name_title = 'Validation Loss', name_axis = 'Loss')

In [ ]:
plot_loss_nets(val_acc, name_title = 'Validation Accuracy', name_axis = 'Acc')

In [ ]:
#ROC_nets(y_true, y_score)